# ENGR-E 533: Deep Learning Systems
## Homework 2

### Khandokar Md. Nayem (knayem@iu.edu)
### Apr 3, 2018

### Import necessary files and set environment parameters
My assigned Node is `r-005` and GPU `0`.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import numpy as np
import librosa
import glob

Using TensorFlow backend.


### Global Variables

Directory names, file formates.

In [25]:
PATH_directory = '/N/u/knayem/data/timit-homework/'

PATH_train = 'tr/'
PATH_val = 'v/'
PATH_test = 'te/'
PATH_denoise = 'dn/'

CLEAN_format_train = 'trs*.wav'
NOISE_format_train = 'trn*.wav'
MIX_format_train = 'trx*.wav'

CLEAN_format_val = 'vs*.wav'
NOISE_format_val = 'vn*.wav'
MIX_format_val = 'vx*.wav'

MIX_format_test = 'tex*.wav'

Max_RNN = 5 # Number of Time Stamps in a RNN 

### File Names

Since there are huge number of files for both training, validation and testing, for convinience we write them in files. These are the name of the file.

In [3]:
TRAIN_S = 'TRAIN_S.txt'
TRAIN_N = 'TRAIN_N.txt'
TRAIN_X_cmplx = 'TRAIN_X_cmplx.txt'
TRAIN_X = 'TRAIN_X.txt'

VAL_S = 'VAL_S.txt'
VAL_S_cmplx = 'VAL_S_cmplx.txt'
VAL_N = 'VAL_N.txt'
VAL_X_cmplx = 'VAL_X_cmplx.txt'
VAL_X = 'VAL_X.txt'


TEST_S = 'TEST_S.txt'
TEST_N = 'TEST_N.txt'
TEST_X_cmplx = 'TEST_X_cmplx.txt'
TEST_X = 'TEST_X.txt'


### Load a .wav file
A function to get the Complex and Magnitude Spectum of a file. 

In [4]:
def preprossed_data(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    X_mag = np.abs(X)
#     print('X_mag',X_mag.shape)
    return X, X_mag

### Save Train Data  

Here we open the files, and read the Train Dataset. We need to run this portion for the first time only. This file reading needs lots fo time. After saving in file once, we can load data from files unless you want complex version of Noisy Speech. Complex numbers are hard to retrive from files.

In [5]:
DATA_train_s = []
DATA_train_n = []
DATA_train_x = []
DATA_train_x_cmplx = []

with open(TRAIN_S,'wb') as fs, open(TRAIN_N,'wb') as fn, open(TRAIN_X_cmplx,'wb') as fx_cmplx, open(TRAIN_X,'wb') as fx: 
    count = 1
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_train+CLEAN_format_train)),sorted(glob.glob(PATH_directory+PATH_train+NOISE_format_train)),sorted(glob.glob(PATH_directory+PATH_train+MIX_format_train))):
        _,s = preprossed_data(file_s)
        DATA_train_s.append(np.array(s))
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        _,n = preprossed_data(file_n)
        DATA_train_n.append(np.array(n))
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        DATA_train_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_train_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count, '...')
        count += 1
        
fs.close()
fn.close()
fx_cmplx.close()
fx.close()

1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...
10 ...
11 ...
12 ...
13 ...
14 ...
15 ...
16 ...
17 ...
18 ...
19 ...
20 ...
21 ...
22 ...
23 ...
24 ...
25 ...
26 ...
27 ...
28 ...
29 ...
30 ...
31 ...
32 ...
33 ...
34 ...
35 ...
36 ...
37 ...
38 ...
39 ...
40 ...
41 ...
42 ...
43 ...
44 ...
45 ...
46 ...
47 ...
48 ...
49 ...
50 ...
51 ...
52 ...
53 ...
54 ...
55 ...
56 ...
57 ...
58 ...
59 ...
60 ...
61 ...
62 ...
63 ...
64 ...
65 ...
66 ...
67 ...
68 ...
69 ...
70 ...
71 ...
72 ...
73 ...
74 ...
75 ...
76 ...
77 ...
78 ...
79 ...
80 ...
81 ...
82 ...
83 ...
84 ...
85 ...
86 ...
87 ...
88 ...
89 ...
90 ...
91 ...
92 ...
93 ...
94 ...
95 ...
96 ...
97 ...
98 ...
99 ...
100 ...
101 ...
102 ...
103 ...
104 ...
105 ...
106 ...
107 ...
108 ...
109 ...
110 ...
111 ...
112 ...
113 ...
114 ...
115 ...
116 ...
117 ...
118 ...
119 ...
120 ...
121 ...
122 ...
123 ...
124 ...
125 ...
126 ...
127 ...
128 ...
129 ...
130 ...
131 ...
132 ...
133 ...
134 ...
135 ...
136 ...
137 ...
138 ...
139 

1035 ...
1036 ...
1037 ...
1038 ...
1039 ...
1040 ...
1041 ...
1042 ...
1043 ...
1044 ...
1045 ...
1046 ...
1047 ...
1048 ...
1049 ...
1050 ...
1051 ...
1052 ...
1053 ...
1054 ...
1055 ...
1056 ...
1057 ...
1058 ...
1059 ...
1060 ...
1061 ...
1062 ...
1063 ...
1064 ...
1065 ...
1066 ...
1067 ...
1068 ...
1069 ...
1070 ...
1071 ...
1072 ...
1073 ...
1074 ...
1075 ...
1076 ...
1077 ...
1078 ...
1079 ...
1080 ...
1081 ...
1082 ...
1083 ...
1084 ...
1085 ...
1086 ...
1087 ...
1088 ...
1089 ...
1090 ...
1091 ...
1092 ...
1093 ...
1094 ...
1095 ...
1096 ...
1097 ...
1098 ...
1099 ...
1100 ...
1101 ...
1102 ...
1103 ...
1104 ...
1105 ...
1106 ...
1107 ...
1108 ...
1109 ...
1110 ...
1111 ...
1112 ...
1113 ...
1114 ...
1115 ...
1116 ...
1117 ...
1118 ...
1119 ...
1120 ...
1121 ...
1122 ...
1123 ...
1124 ...
1125 ...
1126 ...
1127 ...
1128 ...
1129 ...
1130 ...
1131 ...
1132 ...
1133 ...
1134 ...
1135 ...
1136 ...
1137 ...
1138 ...
1139 ...
1140 ...
1141 ...
1142 ...
1143 ...
1144 ...
1145 ...
1

### Save Validate Data

Here we open the files, and read the Validation Dataset. To evaluate SNR of validation data, we need to run this.

In [5]:
DATA_val_s = []
DATA_val_s_cmplx = []
DATA_val_n = []
DATA_val_x_cmplx = []
DATA_val_x = []

with open(VAL_S,'wb') as fs, open(VAL_N,'wb') as fn, open(VAL_X_cmplx,'wb') as fx_cmplx, open(VAL_X,'wb') as fx, open(VAL_S_cmplx,'wb') as fs_cmplx: 
    count = 1
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)),sorted(glob.glob(PATH_directory+PATH_val+NOISE_format_val)),sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
        s_cmplx,s = preprossed_data(file_s)
        DATA_val_s.append(np.array(s))
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        DATA_val_s_cmplx.append(np.array(s_cmplx))
        np.savetxt(fs_cmplx, s_cmplx, fmt='%.5f')
        fs_cmplx.write(b'\n')
        
        _,n = preprossed_data(file_n)
        DATA_val_n.append(np.array(n))
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        DATA_val_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_val_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count, '...')
        count += 1
        
fs.close()
fs_cmplx.close()
fn.close()
fx_cmplx.close()
fx.close()

1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...
10 ...
11 ...
12 ...
13 ...
14 ...
15 ...
16 ...
17 ...
18 ...
19 ...
20 ...
21 ...
22 ...
23 ...
24 ...
25 ...
26 ...
27 ...
28 ...
29 ...
30 ...
31 ...
32 ...
33 ...
34 ...
35 ...
36 ...
37 ...
38 ...
39 ...
40 ...
41 ...
42 ...
43 ...
44 ...
45 ...
46 ...
47 ...
48 ...
49 ...
50 ...
51 ...
52 ...
53 ...
54 ...
55 ...
56 ...
57 ...
58 ...
59 ...
60 ...
61 ...
62 ...
63 ...
64 ...
65 ...
66 ...
67 ...
68 ...
69 ...
70 ...
71 ...
72 ...
73 ...
74 ...
75 ...
76 ...
77 ...
78 ...
79 ...
80 ...
81 ...
82 ...
83 ...
84 ...
85 ...
86 ...
87 ...
88 ...
89 ...
90 ...
91 ...
92 ...
93 ...
94 ...
95 ...
96 ...
97 ...
98 ...
99 ...
100 ...
101 ...
102 ...
103 ...
104 ...
105 ...
106 ...
107 ...
108 ...
109 ...
110 ...
111 ...
112 ...
113 ...
114 ...
115 ...
116 ...
117 ...
118 ...
119 ...
120 ...
121 ...
122 ...
123 ...
124 ...
125 ...
126 ...
127 ...
128 ...
129 ...
130 ...
131 ...
132 ...
133 ...
134 ...
135 ...
136 ...
137 ...
138 ...
139 

1035 ...
1036 ...
1037 ...
1038 ...
1039 ...
1040 ...
1041 ...
1042 ...
1043 ...
1044 ...
1045 ...
1046 ...
1047 ...
1048 ...
1049 ...
1050 ...
1051 ...
1052 ...
1053 ...
1054 ...
1055 ...
1056 ...
1057 ...
1058 ...
1059 ...
1060 ...
1061 ...
1062 ...
1063 ...
1064 ...
1065 ...
1066 ...
1067 ...
1068 ...
1069 ...
1070 ...
1071 ...
1072 ...
1073 ...
1074 ...
1075 ...
1076 ...
1077 ...
1078 ...
1079 ...
1080 ...
1081 ...
1082 ...
1083 ...
1084 ...
1085 ...
1086 ...
1087 ...
1088 ...
1089 ...
1090 ...
1091 ...
1092 ...
1093 ...
1094 ...
1095 ...
1096 ...
1097 ...
1098 ...
1099 ...
1100 ...
1101 ...
1102 ...
1103 ...
1104 ...
1105 ...
1106 ...
1107 ...
1108 ...
1109 ...
1110 ...
1111 ...
1112 ...
1113 ...
1114 ...
1115 ...
1116 ...
1117 ...
1118 ...
1119 ...
1120 ...
1121 ...
1122 ...
1123 ...
1124 ...
1125 ...
1126 ...
1127 ...
1128 ...
1129 ...
1130 ...
1131 ...
1132 ...
1133 ...
1134 ...
1135 ...
1136 ...
1137 ...
1138 ...
1139 ...
1140 ...
1141 ...
1142 ...
1143 ...
1144 ...
1145 ...
1

### Save Test Data

Here we open the files, and read the Test Dataset. To construct Test output files, we need to run this.

In [6]:
DATA_test_x_cmplx = []
DATA_test_x = []

with open(TEST_S,'wb') as fs, open(TEST_N,'wb') as fn, open(TEST_X_cmplx,'wb') as fx_cmplx, open(TEST_X,'wb') as fx: 
    count = 1
    for file_x in sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test)):
        x_cmplx,x = preprossed_data(file_x)
        DATA_test_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_test_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count,'...')
        count += 1
        
fx_cmplx.close()
fx.close()

1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...
10 ...
11 ...
12 ...
13 ...
14 ...
15 ...
16 ...
17 ...
18 ...
19 ...
20 ...
21 ...
22 ...
23 ...
24 ...
25 ...
26 ...
27 ...
28 ...
29 ...
30 ...
31 ...
32 ...
33 ...
34 ...
35 ...
36 ...
37 ...
38 ...
39 ...
40 ...
41 ...
42 ...
43 ...
44 ...
45 ...
46 ...
47 ...
48 ...
49 ...
50 ...
51 ...
52 ...
53 ...
54 ...
55 ...
56 ...
57 ...
58 ...
59 ...
60 ...
61 ...
62 ...
63 ...
64 ...
65 ...
66 ...
67 ...
68 ...
69 ...
70 ...
71 ...
72 ...
73 ...
74 ...
75 ...
76 ...
77 ...
78 ...
79 ...
80 ...
81 ...
82 ...
83 ...
84 ...
85 ...
86 ...
87 ...
88 ...
89 ...
90 ...
91 ...
92 ...
93 ...
94 ...
95 ...
96 ...
97 ...
98 ...
99 ...
100 ...
101 ...
102 ...
103 ...
104 ...
105 ...
106 ...
107 ...
108 ...
109 ...
110 ...
111 ...
112 ...
113 ...
114 ...
115 ...
116 ...
117 ...
118 ...
119 ...
120 ...
121 ...
122 ...
123 ...
124 ...
125 ...
126 ...
127 ...
128 ...
129 ...
130 ...
131 ...
132 ...
133 ...
134 ...
135 ...
136 ...
137 ...
138 ...
139 

### Load Data

Function to load data from files.

In [7]:
def load_file(file_name):
    
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
                
        return sentence_full

Load the Train files.

In [8]:
DATA_train_s = load_file(TRAIN_S)
DATA_train_n = load_file(TRAIN_N)
DATA_train_x_cmplx = load_file(TRAIN_X_cmplx)
DATA_train_x = load_file(TRAIN_X)

616800
616800
616800
616800


Load the Validation files. Do not run this if you want to calculate SNR.

DATA_val_s = load_file(VAL_S)
DATA_val_n = load_file(VAL_N)
DATA_val_x_cmplx = load_file(VAL_X_cmplx)
DATA_val_x = load_file(VAL_X)

Load the Test files. Do not run this if you want to creat wav output files.

DATA_test_x_cmplx = load_file(TEST_X_cmplx)
DATA_test_x = load_file(TEST_X)

### Training and Validation Labels (IRM)

In [9]:
DATA_train_M=[ (1.0*(DATA_train_s[i]>DATA_train_n[i])) for i in range(len(DATA_train_s)) ]

DATA_val_M=[ (1.0*(DATA_val_s[i]>DATA_val_n[i])) for i in range(len(DATA_val_s)) ]
    

### Batch Generator with Mini-Batch

In [10]:
def next_batch_mb(X_,Y_,mini_batch):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
        if e>0 and (e+1)%10==0:

            batch_x, temp_x = None, batch_x
            batch_y, temp_y = None, batch_y
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))
            
            total_mini_batch = temp_x.shape[0]//mini_batch
     
            for mb in range(total_mini_batch):
                start_b = (mb*mini_batch)
                end_b = ((mb+1)*mini_batch)

                yield temp_x[start_b:end_b],temp_y[start_b:end_b]

    

### Batch Generator

In [11]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)

        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))
        
            yield temp_x,temp_y

We try various model for this problem. And comparing all these, we choose to model 3 as best.

### Model 1: Single Layer RNN (GRU)

### Model 2: Single Layer RNN (GRU) with Mini-batch (10)

### Model 3: Two layer Bi-directional RNN (GRU), droupout=0.2 

In [12]:
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))

model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for ep in range(10):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(DATA_train_x, DATA_train_M), next_batch(DATA_val_x, DATA_val_M)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=100)

        
# Final evaluation of the model
scores = []
for v_x,v_y in next_batch(DATA_val_x, DATA_val_x):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )

print("Accuracy: %.2f%%" % np.mean(scores))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 10)             480       
_________________________________________________________________
dense_1 (Dense)              (None, 5, 513)            5643      
Total params: 21,693
Trainable params: 21,693
Non-trainable params: 0
_________________________________________________________________
None
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 3s 25ms/step - loss: 0.6932 - acc: 0.4995 - val_loss: 0.6928 - val_acc: 0.5173
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [======================

Train on 210 samples, validate on 210 samples
Epoch 1/1
210/210 [==============================] - 0s 640us/step - loss: 0.6817 - acc: 0.5550 - val_loss: 0.6747 - val_acc: 0.5520
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 601us/step - loss: 0.6559 - acc: 0.6348 - val_loss: 0.6699 - val_acc: 0.5613
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 546us/step - loss: 0.6575 - acc: 0.6116 - val_loss: 0.6622 - val_acc: 0.5822
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 693us/step - loss: 0.6854 - acc: 0.5019 - val_loss: 0.6709 - val_acc: 0.5498
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 519us/step - loss: 0.6636 - acc: 0.5790 - val_loss: 0.6570 - val_acc: 0.6021
Train on 214 samples, validate on 214 samples
Epoch 1/1
214/214 [==============================] - 0s 578

Train on 228 samples, validate on 228 samples
Epoch 1/1
228/228 [==============================] - 0s 610us/step - loss: 0.6752 - acc: 0.5580 - val_loss: 0.6511 - val_acc: 0.6313
Train on 202 samples, validate on 202 samples
Epoch 1/1
202/202 [==============================] - 0s 697us/step - loss: 0.6407 - acc: 0.6452 - val_loss: 0.6428 - val_acc: 0.6377
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 577us/step - loss: 0.6512 - acc: 0.6073 - val_loss: 0.6427 - val_acc: 0.6543
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 597us/step - loss: 0.6924 - acc: 0.5582 - val_loss: 0.6875 - val_acc: 0.5742
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 692us/step - loss: 0.6423 - acc: 0.6516 - val_loss: 0.6464 - val_acc: 0.6281
Train on 356 samples, validate on 356 samples
Epoch 1/1
356/356 [==============================] - 0s 558

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 633us/step - loss: 0.6122 - acc: 0.6627 - val_loss: 0.6145 - val_acc: 0.6745
Train on 262 samples, validate on 262 samples
Epoch 1/1
262/262 [==============================] - 0s 541us/step - loss: 0.6689 - acc: 0.5971 - val_loss: 0.6371 - val_acc: 0.6485
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 616us/step - loss: 0.5989 - acc: 0.6965 - val_loss: 0.5693 - val_acc: 0.7415
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 575us/step - loss: 0.6422 - acc: 0.6369 - val_loss: 0.6675 - val_acc: 0.6381
Train on 188 samples, validate on 188 samples
Epoch 1/1
188/188 [==============================] - 0s 536us/step - loss: 0.6081 - acc: 0.6804 - val_loss: 0.6026 - val_acc: 0.6955
Train on 216 samples, validate on 216 samples
Epoch 1/1
216/216 [==============================] - 0s 666

Train on 118 samples, validate on 118 samples
Epoch 1/1
118/118 [==============================] - 0s 891us/step - loss: 0.6172 - acc: 0.6570 - val_loss: 0.6207 - val_acc: 0.6718
Train on 232 samples, validate on 232 samples
Epoch 1/1
232/232 [==============================] - 0s 657us/step - loss: 0.6213 - acc: 0.6643 - val_loss: 0.6210 - val_acc: 0.6778
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 715us/step - loss: 0.5697 - acc: 0.7198 - val_loss: 0.5815 - val_acc: 0.7290
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 658us/step - loss: 0.6366 - acc: 0.6452 - val_loss: 0.6107 - val_acc: 0.6808
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 654us/step - loss: 0.5897 - acc: 0.6947 - val_loss: 0.5689 - val_acc: 0.7176
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 636

...

Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 503us/step - loss: 0.5728 - acc: 0.7057 - val_loss: 0.5397 - val_acc: 0.7295
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 550us/step - loss: 0.5426 - acc: 0.7313 - val_loss: 0.5398 - val_acc: 0.7314
Train on 222 samples, validate on 222 samples
Epoch 1/1
222/222 [==============================] - 0s 700us/step - loss: 0.5422 - acc: 0.7315 - val_loss: 0.5452 - val_acc: 0.7343
Train on 296 samples, validate on 296 samples
Epoch 1/1
296/296 [==============================] - 0s 545us/step - loss: 0.5449 - acc: 0.7319 - val_loss: 0.5320 - val_acc: 0.7484
Train on 230 samples, validate on 230 samples
Epoch 1/1
230/230 [==============================] - 0s 702us/step - loss: 0.5225 - acc: 0.7516 - val_loss: 0.5210 - val_acc: 0.7565
Train on 190 samples, validate on 190 samples
Epoch 1/1
190/190 [==============================] - 0s 495

Train on 234 samples, validate on 234 samples
Epoch 1/1
234/234 [==============================] - 0s 633us/step - loss: 0.5051 - acc: 0.7624 - val_loss: 0.4700 - val_acc: 0.7872
Train on 184 samples, validate on 184 samples
Epoch 1/1
184/184 [==============================] - 0s 533us/step - loss: 0.5216 - acc: 0.7572 - val_loss: 0.5138 - val_acc: 0.7590
Train on 238 samples, validate on 238 samples
Epoch 1/1
238/238 [==============================] - 0s 782us/step - loss: 0.5669 - acc: 0.7120 - val_loss: 0.5074 - val_acc: 0.7562
Train on 340 samples, validate on 340 samples
Epoch 1/1
340/340 [==============================] - 0s 560us/step - loss: 0.5436 - acc: 0.7336 - val_loss: 0.5144 - val_acc: 0.7531
Train on 220 samples, validate on 220 samples
Epoch 1/1
220/220 [==============================] - 0s 670us/step - loss: 0.5294 - acc: 0.7517 - val_loss: 0.5344 - val_acc: 0.7442
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [==============================] - 0s 603

Train on 176 samples, validate on 176 samples
Epoch 1/1
176/176 [==============================] - 0s 517us/step - loss: 0.5243 - acc: 0.7444 - val_loss: 0.5416 - val_acc: 0.7304
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 722us/step - loss: 0.5205 - acc: 0.7446 - val_loss: 0.5242 - val_acc: 0.7413
Train on 330 samples, validate on 330 samples
Epoch 1/1
330/330 [==============================] - 0s 604us/step - loss: 0.5645 - acc: 0.7072 - val_loss: 0.5663 - val_acc: 0.7061
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 501us/step - loss: 0.5743 - acc: 0.6960 - val_loss: 0.4952 - val_acc: 0.7659
Train on 256 samples, validate on 256 samples
Epoch 1/1
256/256 [==============================] - 0s 537us/step - loss: 0.5422 - acc: 0.7245 - val_loss: 0.4956 - val_acc: 0.7695
Train on 264 samples, validate on 264 samples
Epoch 1/1
264/264 [==============================] - 0s 535

Train on 194 samples, validate on 194 samples
Epoch 1/1
194/194 [==============================] - 0s 491us/step - loss: 0.5245 - acc: 0.7401 - val_loss: 0.5064 - val_acc: 0.7628
Train on 164 samples, validate on 164 samples
Epoch 1/1
164/164 [==============================] - 0s 578us/step - loss: 0.5744 - acc: 0.7009 - val_loss: 0.5360 - val_acc: 0.7302
Train on 346 samples, validate on 346 samples
Epoch 1/1
346/346 [==============================] - 0s 627us/step - loss: 0.5467 - acc: 0.7235 - val_loss: 0.5260 - val_acc: 0.7423
Train on 172 samples, validate on 172 samples
Epoch 1/1
172/172 [==============================] - 0s 609us/step - loss: 0.5308 - acc: 0.7507 - val_loss: 0.5379 - val_acc: 0.7475
Train on 286 samples, validate on 286 samples
Epoch 1/1
286/286 [==============================] - 0s 502us/step - loss: 0.4922 - acc: 0.7719 - val_loss: 0.4950 - val_acc: 0.7687
Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 471

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 600us/step - loss: 0.5635 - acc: 0.7122 - val_loss: 0.5453 - val_acc: 0.7456
Train on 166 samples, validate on 166 samples
Epoch 1/1
166/166 [==============================] - 0s 571us/step - loss: 0.5231 - acc: 0.7416 - val_loss: 0.5027 - val_acc: 0.7694
Train on 224 samples, validate on 224 samples
Epoch 1/1
224/224 [==============================] - 0s 632us/step - loss: 0.5196 - acc: 0.7466 - val_loss: 0.4952 - val_acc: 0.7667
Train on 294 samples, validate on 294 samples
Epoch 1/1
294/294 [==============================] - 0s 519us/step - loss: 0.5374 - acc: 0.7405 - val_loss: 0.5149 - val_acc: 0.7554
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 625us/step - loss: 0.4987 - acc: 0.7586 - val_loss: 0.4854 - val_acc: 0.7774
Train on 196 samples, validate on 196 samples
Epoch 1/1
196/196 [==============================] - 0s 491

### Batch Generator for SNR 

In [13]:
def next_batch_SXX_cmplxS_cmplx(S_,X_,X_cmplx_,S_cmplx_):
    
    batch_s = None
    batch_x = None
    batch_x_cmplx = None
    batch_s_cmplx = None
    
    for e,(s,x,x_cmplx,s_cmplx) in enumerate( zip(S_,X_,X_cmplx_,S_cmplx_)): 
        batch_s = np.array(s.T) if batch_s is None else np.concatenate( (batch_s,s.T), axis=0)
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_x_cmplx = np.array(x_cmplx.T) if batch_x_cmplx is None else np.concatenate( (batch_x_cmplx,x_cmplx.T), axis=0)
        batch_s_cmplx = np.array(s_cmplx.T) if batch_s_cmplx is None else np.concatenate( (batch_s_cmplx,s_cmplx.T), axis=0)
        
        
        if e>0 and (e+1)%10==0:
            temp_s, batch_s = batch_s, None
            temp_x, batch_x = batch_x, None
            temp_x_cmplx, batch_x_cmplx = batch_x_cmplx, None
            temp_s_cmplx, batch_s_cmplx = batch_s_cmplx, None
            
            temp_s = temp_s.reshape((-1,Max_RNN,513))
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_x_cmplx = temp_x_cmplx.reshape((-1,Max_RNN,513))

            yield temp_s,temp_x,temp_x_cmplx,temp_s_cmplx

### Calculate SNR

In [14]:
sum_s = 0.0
sum_s_diff = 0.0


for v_s,v_x,v_x_cmplx,v_s_cmplx in next_batch_SXX_cmplxS_cmplx(DATA_val_s,DATA_val_x,DATA_val_x_cmplx,DATA_val_s_cmplx):
    
    mask = model.predict(v_x)
    S_hat = (mask) * v_x_cmplx
    S_hat = S_hat.reshape(-1,513).T
    S = v_s_cmplx.T

    S_org = librosa.istft(S, hop_length=512)
    S_pred = librosa.istft(S_hat, hop_length=512)

    sum_s += np.sum(S_org*S_org)
    sum_s_diff += np.sum((S_org-S_pred)*(S_org-S_pred))
    
acc = sum_s/ sum_s_diff
print( 'SNR:', 10*np.log10(acc) )

SNR: 5.717474456837075


### Write Audio files

In [35]:
def write_audio():
    mags = None
    cmplxs = None
    g_counter = 0
    
    for e, file_x in enumerate(sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test))):
        print(e)
        sn, sr = librosa.load(file_x, sr=None)
        Sn = librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        
        mags = np.array(mag_Sn.T) if mags is None else np.concatenate( (mags,mag_Sn.T), axis=0)
        cmplxs = np.array(Sn.T) if cmplxs is None else np.concatenate( (cmplxs,Sn.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp, mags = mags, None
            temp_cmplxs, cmplxs = cmplxs, None
            
            temp = temp.reshape((-1,Max_RNN,513))
            mask = model.predict(temp)
            
            mask=mask.reshape(-1,513)
            S_hat = (mask) * temp_cmplxs
            S_hat = S_hat.T
            
            lenght_w = S_hat.shape[1]//10
            for clip in range(10):
                start_w = clip*lenght_w
                end_w = (clip+1)*lenght_w
                
                wav = S_hat[:,start_w:end_w]

                S_time=librosa.istft(wav, hop_length=512)
                fname = PATH_directory+PATH_denoise+ str(g_counter) + "_redoise.wav"
                g_counter += 1
                
                print(fname)
                librosa.output.write_wav(fname, S_time, sr)
                

In [36]:
write_audio()

0
1
2
3
4
5
6
7
8
9
/N/u/knayem/data/timit-homework/dn/0_redoise.wav
/N/u/knayem/data/timit-homework/dn/1_redoise.wav
/N/u/knayem/data/timit-homework/dn/2_redoise.wav
/N/u/knayem/data/timit-homework/dn/3_redoise.wav
/N/u/knayem/data/timit-homework/dn/4_redoise.wav
/N/u/knayem/data/timit-homework/dn/5_redoise.wav
/N/u/knayem/data/timit-homework/dn/6_redoise.wav
/N/u/knayem/data/timit-homework/dn/7_redoise.wav
/N/u/knayem/data/timit-homework/dn/8_redoise.wav
/N/u/knayem/data/timit-homework/dn/9_redoise.wav
10
11
12
13
14
15
16
17
18
19
/N/u/knayem/data/timit-homework/dn/10_redoise.wav
/N/u/knayem/data/timit-homework/dn/11_redoise.wav
/N/u/knayem/data/timit-homework/dn/12_redoise.wav
/N/u/knayem/data/timit-homework/dn/13_redoise.wav
/N/u/knayem/data/timit-homework/dn/14_redoise.wav
/N/u/knayem/data/timit-homework/dn/15_redoise.wav
/N/u/knayem/data/timit-homework/dn/16_redoise.wav
/N/u/knayem/data/timit-homework/dn/17_redoise.wav
/N/u/knayem/data/timit-homework/dn/18_redoise.wav
/N/u/knaye

161
162
163
164
165
166
167
168
169
/N/u/knayem/data/timit-homework/dn/160_redoise.wav
/N/u/knayem/data/timit-homework/dn/161_redoise.wav
/N/u/knayem/data/timit-homework/dn/162_redoise.wav
/N/u/knayem/data/timit-homework/dn/163_redoise.wav
/N/u/knayem/data/timit-homework/dn/164_redoise.wav
/N/u/knayem/data/timit-homework/dn/165_redoise.wav
/N/u/knayem/data/timit-homework/dn/166_redoise.wav
/N/u/knayem/data/timit-homework/dn/167_redoise.wav
/N/u/knayem/data/timit-homework/dn/168_redoise.wav
/N/u/knayem/data/timit-homework/dn/169_redoise.wav
170
171
172
173
174
175
176
177
178
179
/N/u/knayem/data/timit-homework/dn/170_redoise.wav
/N/u/knayem/data/timit-homework/dn/171_redoise.wav
/N/u/knayem/data/timit-homework/dn/172_redoise.wav
/N/u/knayem/data/timit-homework/dn/173_redoise.wav
/N/u/knayem/data/timit-homework/dn/174_redoise.wav
/N/u/knayem/data/timit-homework/dn/175_redoise.wav
/N/u/knayem/data/timit-homework/dn/176_redoise.wav
/N/u/knayem/data/timit-homework/dn/177_redoise.wav
/N/u/k

311
312
313
314
315
316
317
318
319
/N/u/knayem/data/timit-homework/dn/310_redoise.wav
/N/u/knayem/data/timit-homework/dn/311_redoise.wav
/N/u/knayem/data/timit-homework/dn/312_redoise.wav
/N/u/knayem/data/timit-homework/dn/313_redoise.wav
/N/u/knayem/data/timit-homework/dn/314_redoise.wav
/N/u/knayem/data/timit-homework/dn/315_redoise.wav
/N/u/knayem/data/timit-homework/dn/316_redoise.wav
/N/u/knayem/data/timit-homework/dn/317_redoise.wav
/N/u/knayem/data/timit-homework/dn/318_redoise.wav
/N/u/knayem/data/timit-homework/dn/319_redoise.wav
320
321
322
323
324
325
326
327
328
329
/N/u/knayem/data/timit-homework/dn/320_redoise.wav
/N/u/knayem/data/timit-homework/dn/321_redoise.wav
/N/u/knayem/data/timit-homework/dn/322_redoise.wav
/N/u/knayem/data/timit-homework/dn/323_redoise.wav
/N/u/knayem/data/timit-homework/dn/324_redoise.wav
/N/u/knayem/data/timit-homework/dn/325_redoise.wav
/N/u/knayem/data/timit-homework/dn/326_redoise.wav
/N/u/knayem/data/timit-homework/dn/327_redoise.wav
/N/u/k